# Topic Modeling - Artikel Kompas dari tahun 2019 hingga 2020

- Pada studi kasus saat ini yaitu melakukan proses topic modeling, dimana data diambil dari kaggle. dataset sendiri merupakan artikel yang berasal dari salah satu platform berita indonesia yaitu kompas, dimana artikel berita dalam rentang waktu 2019-2020. karena volume data yang terlalu besar sehingga sangat sulit untuk mengehtahui topik utama yang sedang tren pada tahun 2019 - 2020, sulit menyaring informasi yang relavan secara cepat. Untuk itu topic modeling disini menjadi sebuah solusi untuk mengehtahui struktur atau makna yang tersembunyi dalam sebuah dokumen.

- Fokus analisis
  - apa saja kata kunci yang paling mewakili setiap topik?
  - melakukan perbandingan model antara LDA dan BERTopic

In [37]:
# load data
import pandas as pd
import numpy as np

# preprocessing
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import ast

# Model & Visualization
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import matplotlib.pyplot as plt
from bertopic import BERTopic
from sklearn.decomposition import PCA
from gensim.corpora.dictionary import Dictionary
from gensim.models import CoherenceModel
from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabasz_score
from sklearn.metrics import davies_bouldin_score
import pyLDAvis
import pyLDAvis.gensim_models
from gensim.models.ldamodel import LdaModel
from gensim import corpora
from umap import UMAP
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory


In [2]:
# melakukana proses import data dari file csv
data = pd.read_csv('Cache/kompas_news_2019_2020_filtered.csv')

**Noted**

- untuk proses pengumpulan data dilakukan dengan cara mendownload data melalaui Kaggle, dimana data berisi mengenai artikel yang berasal dari kompas muali tahun 2019 - 2020. untuk link datasetnya yaitu sebagai berikut: https://www.kaggle.com/datasets/iwanmanurung/kompas-news-2019-2020/data

In [3]:
# melihat 5 data teratas
data.head()

,date,title,news
0,2019-09-01,Beasiswa S1 plus Tunjangan Bulanan dari Univer...,- chulalongkorndimembuka programsarjana (s1) b...
1,2019-09-01,"""Puldapia Education Expo"" Tampilkan Wajah Dina...","- berdasarkan data kementerian agama, jumlahdi..."
2,2019-09-01,Beasiswa DataPrint untuk 225 Pelajar dan Mahas...,dataprint kembali memberikan programkepada kon...
3,2019-09-01,"5 Wisudawan UI Raih IPK Sempurna, Siapa Mereka?",",universitas indonesia () kembali menyelenggar..."
4,2019-09-01,Mengenal Integrasi Konsep Kampus dan Properti ...,- sektortelah menjadi bagian tidak terpisahaka...


In [4]:
# nelihat total keseluruhan data (baris, kolom dataset)
data.shape

(294955, 3)

In [5]:
# melihat data null yangada pada dataset
data.isna().sum()

date     0
title    0
news     0
dtype: int64

**Noted**

berdasarkan code yang telah dibuat terdapat beberapa proses yang dilakukan, pertama tentunya melakukan proses import data csv yang terdapat pada folder Cache. Langkah kedua adalah meilihat jumlah data secara keselurahan dimana terdapat 294955 baris dan 3 kolom data. Langkah ketiga yaitu melihat apakah pada dataset terdapat data null atau tidal, berdasarkan hasil yang telah dilakukan sebelumnya tidak terdapat data null pada dataset.

In [6]:
# mengambil 50k dataset saja
df_sample = data.sample(n=50000, random_state=42)

## Preprocessing Data

pemrosesan utama data terdiri dari case folding, tokenisasi, stopword removal, dan filtering kata yang tidak penting.

In [7]:
# menghapus kolom yang tidak diperlukan
df = df_sample.drop(columns=['title'])

In [8]:
# melakukan pengecekan terdahap data suplikat
df.duplicated().sum()

4

In [9]:
df[df.duplicated()]

,date,news
52599,2019-10-26,-menjadi salah satu pilihan minuman yang digem...
204945,2019-08-06,"kh maimun zubair, dikenal luas sebagai mbah mo..."
198998,2019-08-15,"bagi yang tak mencoba mengenali lebih dalam, m..."
94988,2020-01-24,"disebut punya pangsa 99 persen usaha, tapi kon..."


In [10]:
# melakukan pemrosesan data case folding
df['news_case'] = df['news'].str.lower()

**Noted**

Fungsi dari code diatas adalah mengubah semua huruf yang ada di dalam dataset kolom news menjadi huruf kecil, kemudian dimasukkan kedalam kolom baru yaitu news_case

In [11]:
# remove function words
df['news_remove'] = df['news_case'].apply(lambda x: re.sub(r'[^\w\s]', ' ', x))

**Noted**

Proses berikutnya yaitu melakukan pemrosesan dengan menghapus simbol simbol yang terdapat dalam teks, dimana simbol yang ada tidak akan digunakan nantinya sehingga simbol tersebut perlu dihapus.

In [12]:
# Pemrosesan data menjadi token
df['news_tokens'] = df['news_remove'].apply(lambda x: re.findall(r'\b[\w-]+\b', x))

**Noted**

pada tahap ini pmeorsesan yang dilakukan adalah membuat setiap baris dataset yang ada menjadi sebuah token, atau biasa juga disebut dengan tokenisasi.

In [13]:
df.head()

,date,news,news_case,news_remove,news_tokens
36198,2020-04-26,", , sejumlah di jawa tengah memilih untuk ti...",", , sejumlah di jawa tengah memilih untuk ti...",sejumlah di jawa tengah memilih untuk ti...,"[sejumlah, di, jawa, tengah, memilih, untuk, t..."
60401,2020-03-05,- polres metro jakarta utara menjual ribuan ma...,- polres metro jakarta utara menjual ribuan ma...,polres metro jakarta utara menjual ribuan ma...,"[polres, metro, jakarta, utara, menjual, ribua..."
226017,2019-03-30,- Pertandingan Persebaya Surabaya versus PS T...,- pertandingan persebaya surabaya versus ps t...,pertandingan persebaya surabaya versus ps t...,"[pertandingan, persebaya, surabaya, versus, ps..."
225869,2019-03-29,- Maskapai Garuda Indonesia akan memberikan d...,- maskapai garuda indonesia akan memberikan d...,maskapai garuda indonesia akan memberikan d...,"[maskapai, garuda, indonesia, akan, memberikan..."
154977,2019-12-10,",timnas u23 indonesia kalah 0-3 dari timnas u2...",",timnas u23 indonesia kalah 0-3 dari timnas u2...",timnas u23 indonesia kalah 0 3 dari timnas u2...,"[timnas, u23, indonesia, kalah, 0, 3, dari, ti..."


In [14]:
# melakukan proses stopword removal
stop_words = set(stopwords.words('indonesian'))
stopword_costume = ['chulalongkorndimembuka', 'sektortelah', 'programkepada ']
stop_words.update(stopword_costume)
def remove_stopwords(tokens):
    return [word for word in tokens if word not in stop_words]

In [15]:
df['news_stopwords'] = df['news_tokens'].apply(remove_stopwords)

**Noted**

pada proses ini yaitu melakukan filtering setiap kata yang ada, kata yang tidak memiliki makna akan dihapus. terdapat juga kata yang saya hapus secara manual dimana data tersebut terdapat kesalahan dalam penulisan. selain itu menurut pendapat saya, kata tersebut juga tidak terlalu memiliki makna yang signifikan terdapat data yang ada.

In [16]:
df.head()

,date,news,news_case,news_remove,news_tokens,news_stopwords
36198,2020-04-26,", , sejumlah di jawa tengah memilih untuk ti...",", , sejumlah di jawa tengah memilih untuk ti...",sejumlah di jawa tengah memilih untuk ti...,"[sejumlah, di, jawa, tengah, memilih, untuk, t...","[jawa, memilih, memilih, bertahan, tinggalnya,..."
60401,2020-03-05,- polres metro jakarta utara menjual ribuan ma...,- polres metro jakarta utara menjual ribuan ma...,polres metro jakarta utara menjual ribuan ma...,"[polres, metro, jakarta, utara, menjual, ribua...","[polres, metro, jakarta, utara, menjual, ribua..."
226017,2019-03-30,- Pertandingan Persebaya Surabaya versus PS T...,- pertandingan persebaya surabaya versus ps t...,pertandingan persebaya surabaya versus ps t...,"[pertandingan, persebaya, surabaya, versus, ps...","[pertandingan, persebaya, surabaya, versus, ps..."
225869,2019-03-29,- Maskapai Garuda Indonesia akan memberikan d...,- maskapai garuda indonesia akan memberikan d...,maskapai garuda indonesia akan memberikan d...,"[maskapai, garuda, indonesia, akan, memberikan...","[maskapai, garuda, indonesia, diskon, harga, t..."
154977,2019-12-10,",timnas u23 indonesia kalah 0-3 dari timnas u2...",",timnas u23 indonesia kalah 0-3 dari timnas u2...",timnas u23 indonesia kalah 0 3 dari timnas u2...,"[timnas, u23, indonesia, kalah, 0, 3, dari, ti...","[timnas, u23, indonesia, kalah, 0, 3, timnas, ..."


**Noted**

Setelah melakukan beberapa pemrosesan data sebelumnya maka dapat kita lihat hasil dari pemorsesan data yang telah dilakukan, agar terlihat perbedaannya disini saya membuat kolom baru setiap step pemrosesan data yang ada.

In [17]:
# melakukan proses BOW untuk digunakan pada algoritma LDA

# proses penggabungan dataset yang semulanya berupa list menjadi string
df['clean_tokens'] = df['news_stopwords'].apply(lambda words: [w for w in words if w.isalpha()])
text = df['clean_tokens'].apply(lambda x: ' '.join(x))

# melakukan proses initialisasi CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1, 2), min_df=5)

# tahap selanjutnya yaitu fit transform data
bow = vectorizer.fit_transform(text)

# mendapatkan kosakata
vocab = vectorizer.get_feature_names_out()


In [18]:
# melihat kosakata yang ada pada dataset
vocab[:10]

array(['aa', 'aa positif', 'aaa', 'aaji', 'aal', 'aal melewati',
       'aaliyah', 'aaliyah massaid', 'aam', 'aam pbnu'], dtype=object)

**Noted**

proses diatas merupakan proses BOW dimana nantinya akan digunakan pada LDA. BOW sendiri yaitu mengubah sekumpulan teks corpus menjedasi refresentasi vektor berdasarkan frekuensi. Pada studi kasus ini juga saya menggunakan Bigram (1,2), min_df berfungsi untuk menentukan jumlah dokumen yang mengandung suatu kata untuk diikutsertakan ke dalam BOW nantinya.

## Algoritma 

### LDA

In [19]:
# algoritma LDA
model_lda = LatentDirichletAllocation(n_components=15, max_iter=10, random_state=42)
lda = model_lda.fit(bow)

In [20]:
# meilihat Fitur atau topik yang dihasilkan
lda.components_.shape

(15, 236651)

In [21]:
# meanampilkan topik yang dihasilkan oleh model LDA
def topic (model):
     return [[vocab[idx] for idx in reversed(comp.argsort()[-6:]) if vocab[idx].isalnum()]
        for comp in model.components_]

**Noted**

proses diatas merupakan proses untuk melatih model LDA dimana pada pelatihan model tersebut terdapat beberapa parameter yang digunakan yaitu:
- n_components --> berfungsi untuk menentukan jumlah topik yang akan ditemukan oleh LDA
- max_iter --> merupakan jumlah maksimal iterasi yang dilakukan untuk mengoptimalkan hasil topik.
- random_state --> mengatur seed random agar tetap konsisten

selain itu juga terdapat proses dimana menampilkan daftar kata-kata topik utama dari model LDA, terdapat pada fungsi "topic"

### BERTopic

In [22]:
# Ambil stopwords Bahasa Indonesia dari Sastrawi
stopwords_id = StopWordRemoverFactory().get_stop_words()

# Konversi list token menjadi string (dokumen)
docs = df['clean_tokens'].apply(lambda x: ' '.join(x)).tolist()

# UMAP model
umap_model = UMAP(
    n_neighbors=10,
    n_components=5,
    min_dist=0.0,
    metric='cosine',
    low_memory=False,
    random_state=1337
)

In [23]:
# Vectorizer dengan stopwords dari Sastrawi
vectorizer_model = CountVectorizer(
    stop_words=stopwords_id,
    ngram_range=(1, 2),
    min_df=5
)

In [24]:
# BERTopic model
topic_model = BERTopic(
    language="indonesian",
    umap_model=umap_model,
    calculate_probabilities=True,
    vectorizer_model=vectorizer_model,
    verbose=True
)

In [25]:
# Fit model
topics, probs = topic_model.fit_transform(docs)

2025-06-10 06:26:23,668 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 1563/1563 [24:56<00:00,  1.04it/s]
2025-06-10 06:51:33,924 - BERTopic - Embedding - Completed ✓
2025-06-10 06:51:33,926 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-06-10 06:52:54,107 - BERTopic - Dimensionality - Completed ✓
2025-06-10 06:52:54,109 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-06-10 07:08:37,939 - BERTopic - Cluster - Completed ✓
2025-06-10 07:08:37,957 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-06-10 07:09:03,588 - BERTopic - Representation - Completed ✓


In [ ]:
# # menampilkan topik yang dihasilkan olej model BERTopic
# def bert_topics_dict(model, top_n_words=6):
#     topics = model.get_topics()
#     return {
#         topic_id: topics[topic_id][:top_n_words]
#         for topic_id in sorted(topics.keys())
#         if topic_id != -1
#     }

**Noted**

Proses diatas merupakan proses pelatihan model BERTopic, dimana pada saat pelatihan model bahasa disetting menjadi bahasa indonesia, alasannya karena dataset tentunya dalam bahasa indonesia. kemudian terdapat juga proses fungsi untuk menampilkan topic yang didapatkan oleh model BERTtopic
- n_neighbors=10: merupakan jumlah tetangga terdekat saat membuat struktur lokal dimana berpengaruh terhadap struktur dari cluster.
- n_components bentuk vektor akan berbentuk dimensi sesuai nilai yang di inputkan untuk nilai emmbeding
- metric='cosine'--> menggunakan consine similairty untuk melakukan pengukuran terrhadap kedekatan dokumen
- calculate_probabilities --> dibuat menjadi true, berfungsi untuk menghitung probabilitas topik untuk setiap dokumen.
-verbose --> dibuat true, berfungsi untuk mencetak proses informasi training



## Evaluasi dan Perbandingan Hasil

### LDA

In [34]:
# Preprocess token list
texts = df['clean_tokens'].tolist()

# Buat dictionary dan corpus
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [35]:
# LDA model dari Gensim
lda_model = LdaModel(corpus=corpus,
                     id2word=dictionary,
                     num_topics=15,
                     random_state=42,
                     passes=10,
                     alpha='auto',
                     per_word_topics=True)

In [36]:
# Visualisasi dengan pyLDAvis
pyLDAvis.enable_notebook()
panel = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
panel

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
12    -0.038986  0.071672       1        1  11.566777
6     -0.113310  0.059439       2        1  11.067967
1      0.099670  0.048344       3        1  10.621357
8     -0.058147 -0.127516       4        1   7.106228
0      0.237995 -0.137553       5        1   7.041773
3      0.061092  0.237916       6        1   6.968499
7     -0.124820  0.058702       7        1   6.743491
2     -0.164201 -0.142329       8        1   6.649476
14    -0.145277  0.072567       9        1   6.106963
5      0.098517  0.191592      10        1   5.202370
4      0.040049 -0.101027      11        1   4.753487
10     0.101060  0.062401      12        1   4.452897
9      0.278203 -0.139873      13        1   4.311426
13    -0.083359 -0.012507      14        1   4.070812
11    -0.188485 -0.141828      15        1   3.336478, topic_info=           Term          Freq         Total Category  logprob  loglift
195          rp  29576.000000  29576.000000  Default  30.0000  30.0000
33        jalan  21492.000000  21492.000000  Default  29.0000  29.0000
18        covid  16185.000000  16185.000000  Default  28.0000  28.0000
150   indonesia  45339.000000  45339.000000  Default  27.0000  27.0000
126       virus  15166.000000  15166.000000  Default  26.0000  26.0000
..          ...           ...           ...      ...      ...      ...
212     anggota   1416.208626  10870.319187  Topic15  -5.3123   1.3622
164  masyarakat   1496.866358  23478.166960  Topic15  -5.2569   0.6476
37        kamis   1223.272853  12779.925559  Topic15  -5.4587   1.0539
340     menteri   1079.069429  10223.353254  Topic15  -5.5841   1.1517
233       jumat   1068.578233  11929.714599  Topic15  -5.5939   0.9876

[1051 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
4977     14  0.998296      abk
283       1  0.025132    acara
283       3  0.334853    acara
283       4  0.183643    acara
283       7  0.006880    acara
...     ...       ...      ...
2626     10  0.046606  youtube
2626     13  0.010357  youtube
8826     11  0.994091     yuan
4754      7  0.999244     yuri
9658      6  0.998245      zat

[2951 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[13, 7, 2, 9, 1, 4, 8, 3, 15, 6, 5, 11, 10, 14, 12])

### BERTopic

In [ ]:
# # Reduksi dimensi untuk evaluasi
# pca = PCA(n_components=5)
# embeddings_pca = pca.fit_transform(embeddings)

# # Filter out outlier topics (-1, -2)
# valid_idx = [i for i, t in enumerate(topics) if t >= 0]
# filtered_embeddings = embeddings_pca[valid_idx]
# filtered_topics = [topics[i] for i in valid_idx]

# # Hitung Silhouette Score
# score = silhouette_score(filtered_embeddings, filtered_topics)
# print(f"\nSilhouette Score: {score:.4f}")

In [26]:
# Dapatkan embeddings
embeddings = topic_model._extract_embeddings(docs, method="document")

# Reduksi dimensi
pca = PCA(n_components=10)
embeddings_pca = pca.fit_transform(embeddings)

# Evaluasi
print("Silhouette Score:", silhouette_score(embeddings_pca, topics))
print("Calinski-Harabasz:", calinski_harabasz_score(embeddings_pca, topics))
print("Davies-Bouldin:", davies_bouldin_score(embeddings_pca, topics))

Silhouette Score: -0.29867706
Calinski-Harabasz: 59.655736412796756
Davies-Bouldin: 2.822230872515802


### Menampilkan Topik antara LDA dan BERTopic

In [27]:
topic(model_lda)

[['jakarta', 'dki', 'kota', 'surat', 'kpu'],
 ['anak', 'video', 'akun', 'orang', 'media', 'ya'],
 ['laga', 'gol', 'pertandingan', 'liga', 'pemain', 'united'],
 ['menteri', 'indonesia', 'kapal', 'presiden', 'pesawat', 'bandara'],
 ['partai', 'jokowi', 'prabowo', 'presiden', 'makanan', 'ketua'],
 ['orang', 'indonesia', 'as', 'film', 'dunia', 'negara'],
 ['memiliki', 'mobil', 'produk', 'air', 'ponsel', 'fitur'],
 ['persen', 'pemerintah', 'rp', 'indonesia', 'negara', 'masyarakat'],
 ['polisi', 'jakarta', 'kota', 'tersangka', 'polda', 'wilayah'],
 ['jalan', 'kendaraan', 'tol', 'jakarta', 'penumpang', 'bus'],
 ['korban', 'warga', 'rumah', 'pelaku', 'desa', 'kecamatan'],
 ['pemain', 'indonesia', 'tim', 'klub', 'musim', 'timnas'],
 ['covid', 'orang', 'virus', 'corona', 'pasien'],
 ['rp', 'juta', 'harga', 'wisata', 'rumah'],
 ['pasal', 'hukum', 'undang', 'hakim', 'pengadilan', 'pidana']]

In [29]:
# Info topik
topic_info = topic_model.get_topic_info()
print(topic_info.head(10))

# Top kata kunci per topik
for i in range(10):
    print(f"\nTopik {i}:")
    print(topic_model.get_topic(i))



   Topic  Count                                    Name  \
0     -1  24384         -1_orang_indonesia_anak_jakarta   
1      0    727               0_lagu_konser_musik_album   
2      1    536       1_hakim_terdakwa_pengadilan_jaksa   
3      2    504  2_pesawat_penerbangan_bandara_maskapai   
4      3    429       3_film_sutradara_karakter_bioskop   
5      4    358          4_pebalap_rossi_marquez_motogp   
6      5    352                  5_kpu_suara_pemilu_tps   
7      6    304                6_gim_ganda_ahsan_marcus   
8      7    301           7_sepatu_koleksi_busana_warna   
9      8    297             8_banjir_air_hujan_genangan   

                                      Representation  \
0  [orang, indonesia, anak, jakarta, rumah, masya...   
1  [lagu, konser, musik, album, band, penyanyi, p...   
2  [hakim, terdakwa, pengadilan, jaksa, majelis h...   
3  [pesawat, penerbangan, bandara, maskapai, penu...   
4  [film, sutradara, karakter, bioskop, tayang, a...   
5  [pebalap, r

**Noted**

berdasarkan hasil dari topic modelling perbandingan hasil antara LDA dan BERTopic seperti hasil yang terlihat. LDA lebih memberikan hasil yang baik dibandingkan BERTopic. alasannya yaitu karenanilai evaluasi yang didapatkan oleh model BERTopic tidak maksimal dan perlu dilakukan peningkatan lagi. 

dari segi respon topic antara model BERTopic dan LDA. LDA memberikan topic yang mudah untuk dipahami dan difenisikan topic tersebut membahasa tentang apa, sama juga seperti BERTopic topic yang diberikan masih cuku mudah untuk dipahami dan didefenisikan.

## Kesimpulan

- Berdasarkan hal yang telah dijelaskan sebelumnya model LDA mempu menghasilkan topic yang lebih baik dibandingkan model BERTopic hal ini dapat dilihat dari hasil evaluasi yang telah dilakukan. terutama pada perbandingan hasil topic yang dihasilkan.

- keunggulan LDA yaitu cendrung ringan, mampu menangani data yang besar, setiap topik yang ada didasarkan dari distribusi kata.
- keunggulan BERTopic adalah mudah dalam menyatukan topic topic yag mirip, memiliki sifat yang lebih fleksibel
- untuk topic yang sering muncul dari kedua model yang digunakan yaitu FILM, Politic, Sport, transportasi, teknologi

# Tambahan

In [38]:
import joblib

# Simpan model LDA ke file
joblib.dump(lda, 'Util/lda_model.pkl')

['Util/lda_model.pkl']

In [39]:
from scipy.sparse import save_npz


save_npz("Util/bow_matrix.npz", bow)
joblib.dump(vectorizer, "Util/count_vectorizer.pkl")
df.to_csv("Data/dataset.csv", index=False, encoding='utf-8')
